# Extracting Gendered Constructions

This Notebook will ask a high-level, guiding question about gender: what kinds of actions are associated with the pronoun "he" and the pronoun "she."

also include adjective - noun pairs

In [89]:
import spacy, warnings; warnings.simplefilter('ignore')
import csv
import re

In [90]:
nlp = spacy.load('en_core_web_sm')

In [100]:
fname = '/scratch/group/pract-txt-mine/sbuongiorno/hansard_decades/hansard_1800.csv'

with open(fname, newline='') as f:
    reader = csv.reader(f)
    a = list(reader)[1:]
    
data = a[:1000000]

In [92]:
data[:5]

[['S1V0001P0_0',
  'moved that Lord Walsingham be appointed chairman of the committee of privileges for the present session.'],
 ['S1V0001P0_11',
  "The subject, in his contemplation, was such as not only that house, but many thousands of his Majesty's subjects must regard with the utmost concern."],
 ['S1V0001P0_22',
  "He agreed with the noble earl as to the great importance of the subject; it was one therefore which naturally attracted the serious attention of his Majesty's government: their lordships, however, would be at the same time aware of the complexity and intricate nature of the general subject, the variety of details which it embraced, and the correspondent difficulty of forming adequate regulations."],
 ['S1V0001P0_33',
  'He was convinced, for his own part, that the contrary was the fact; and, though die abuse had not been flagrant or excessive, that the Bank had yielded, in a certain degree, to the temptation, and extended the quantity of their notes beyond the proper l

In [83]:
data = [re.sub(r'\b[A-Z]+(?:\s+[A-Z]+)*\b', '', ls) for ls in data] # remove words that are all upper case - so names 
data = [re.sub(r'\\\\n|\\\\t|\'s', '', ls) for ls in data] # remove lien breaks, tab breaks, and possessive "s"
data = [re.sub(r'[^\w\s]|_', '', ls) for ls in data] # remove punctuation and underscore
data = [re.sub(r'\d{1, 3}', '', ls) for ls in data] # remove digits that are a minimum of 1 and a maximum of 3
data = [re.sub(r'\w*\d\w*', '', ls) for ls in data] # remove character strings that contain a digit
            
data = [word.lower() for word in data]

#data = ' '.join(data)

data = [ls for ls in data]

In [101]:
%%time
out = []

for s in data:
    for item in s:
        doc = nlp(item)
        out.append(doc)

KeyboardInterrupt: 

In [55]:
spacy_tokens = []
for token in doc:
    row = token.text, token.lemma_, token.pos_, token.dep_, token.head.text
    spacy_tokens.append(row)

spacy_tokens[:10]

[(' ', ' ', 'SPACE', '', 'moved'),
 ('moved', 'move', 'VERB', 'ROOT', 'moved'),
 ('that', 'that', 'SCONJ', 'mark', 'appointed'),
 ('lord', 'lord', 'PROPN', 'compound', 'walsingham'),
 ('walsingham', 'walsingham', 'PROPN', 'nsubjpass', 'appointed'),
 ('be', 'be', 'AUX', 'auxpass', 'appointed'),
 ('appointed', 'appoint', 'VERB', 'ccomp', 'moved'),
 ('chairman', 'chairman', 'NOUN', 'oprd', 'appointed'),
 ('of', 'of', 'ADP', 'prep', 'chairman'),
 ('the', 'the', 'DET', 'det', 'committee')]

In [31]:
from spacy.symbols import nsubj, nsubjpass, VERB

pairs = []
for subject in doc:
    if subject.dep == nsubj or subject.dep == nsubjpass and subject.head.pos == VERB:
        pairs.append(str(' '.join([subject.text, subject.head.lemma_])))

In [33]:
pairs[:10]

['walsingham appoint',
 'subject be',
 'thousands regard',
 'he agree',
 'it be',
 'which attract',
 'lordships be',
 'it embrace',
 'he convince',
 'contrary be']

In [36]:
stopwords = ['right', 'hon', 'general', 'mr', 'shall', 'majesty', 'exchequer', 'address', 'motion', 'bill',
             'earl', 'friend', 'chancellor', 'sense', 'object', 'suppose', 'amidst', 'noble', 'lord', 'agree',
             'speech', 'kind', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september',
             'october', 'november', 'december', 'order', 'held', 'hold', 'assembly', '£', 'humble', 'about', 'above',
             'across', 'address', 'after', 'afterwards', 'again', 'against', 'agree', 'all', 'almost', 'alone', 'along',
             'already', 'also', 'although', 'always', 'am', 'amidst', 'among', 'amongst', 'amount', 'an', 'and', 'another',
             'any', 'anyhow', 'anything', 'anyway', 'anywhere', 'april', 'are', 'around', 'as', 'at', 'back', 'be', 'was',
             'becames', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being',
             'beside', 'besides', 'between', 'beyond', 'both', 'bottom', 'but', 'by', 'call', 'can', 'cannot', 'do',
             'could', 'did', 'does', 'doing', 'done', 'down', 'due', 'during', 'each', 'either', 'else', 'elsewhere',
             'empty', 'enough', 'even', 'ever', 'everyone', 'everything', 'everywhere', 'except', 'few', 'first', 'for',
             'from', 'front', 'full', 'further', 'general', 'get', 'give', 'go', 'had', 'has', 'have', 'held', 'hence',
             'here', 'hereby', 'herein', 'hold', 'however', 'if', 'in', 'indeed', 'into', 'is', 'it', 'its', 'itself',
             'just', 'keep', 'kind', 'last', 'latter', 'least', 'less', 'made', 'make', 'many', 'me', 'meanwhile', 'might',
             'mine', 'more', 'most', 'mostly', 'move', 'much', 'must', 'name,' 'neither', 'never', 'nevertheless', 'next',
             'no', 'noble', 'nobody', 'none', 'nor', 'nothing', 'now', 'nowhere', 'of', 'off', 'often', 'on', 'one', 'once',
             'only', 'onto', 'or', 'other', 'others', 'otherwise', 'out', 'over', 'own', 'part', 'perhaps', 'please', 'put',
             'quite', 'rather', 'really', 'regarding', 'same', 'say', 'see', 'seem', 'seemed', 'seeming', 'seems', 'sense',
             'several', 'shall', 'should', 'show', 'side', 'since', 'so', 'some', 'someone', 'something', 'sometime',
             'still', 'such', 'suppose', 'take', 'than', 'that', 'the', 'then', 'there', 'thereby', 'therefore', 'these',
             'they', 'this', 'those', 'though', 'thus', 'to', 'together', 'too', 'top', 'toward', 'towards', 'under',
             'unless', 'until', 'up', 'upon', 'us', 'used', 'using', 'various', 'very', 'was', 'we', 'well', 'were', 'what',
             'whatever', 'when', 'when', 'whereby', 'whether', 'which', 'while', 'who', 'why', 'will', 'with', 'within',
             'without', 'would', 'yet', 'bright', 'mr.', 'hansard', 'lancashire', '[]', '£1', '000', 'russell', 'committee',
             'reading', 'learned', 'deal', 'time', 'royal', 'gentlemen', 'gentleman', 'year', 'years', 'affairs', 'affair',
             'academy', 'sir', 'thought', 'took', 'bring', 'brings', 'brought', 'forward', 'great', 'good', 'department',
             'treasury', 'second', 'take', 'taken', 'privy', 'member', 'robert', 'large', 'session', 'secretary', 'notice',
             'moment', 'think', 'believe', 'hope', 'ask', 'hear', 'beg', 'support', 'state', 'find', 'admit', 'wish',
             'refer', 'reply', 'know', 'feel', 'propose', 'understand', 'let', 'allow', 'like', 'receive', 'consider',
             'begin', 'tell', 'like', 'send', 'ought', 'come', 'intend', 'add', 'want', 'stand', 'suggest', 'remind',
             'use', 'mean', 'suggest', 's']


In [37]:
keep_pairs = []
for pair in pairs:
    tokens = pair.split(' ')
    joined_tokens = ' '.join([word for word in tokens if not word in stopwords])
    if len(tokens_filtered) == 2: # if string is length of two
        keep_pairs.append(str(joined_tokens)) # keep the string

NameError: name 'tokens_filtered' is not defined

In [ ]:
regex = re.compile('he ')
male = [word for word in pairs if regex.match(word)]

male[:10]

In [ ]:
def countWords(clean_data):
    ngrams_dict = {}
    for ngram in clean_data:
        if ngram in ngrams_dict:
            ngrams_dict[ngram] += 1
        else:
            ngrams_dict[ngram] = 1
    return ngrams_dict
            
male_dictionary = countWords(male_no_stopwords)

In [ ]:
for kv in list(male_dictionary)[:30]:
    print(kv,  male_dictionary[kv])

Now get vizcode